In [1]:
from bs4 import BeautifulSoup as bs
import lxml
import os
import json

In [66]:
def extractPersonData(filepath):
    
    person_dict = {}
    
    with open(filepath,"r") as file:
        content = file.read()
        bs_content = bs(content, "lxml")
        
        # extract id 
        # (could cut this down to just an id number)
        try:
            person_dict['id'] = bs_content.person["xml:id"]
        except:
            person_dict['id'] = None
            
        # extract sex
        try: 
            sex = bs_content.sex["value"]
            if sex.upper() == 'M':
                person_dict['sex'] = 'M'
            elif sex.upper() in ['W', 'F']:
                person_dict['sex'] = 'F'
            else:
                person_dict['sex'] = ''
        except:
            person_dict['sex'] = ''
            
        # extract name
        names = []
        try:
            forenames = [
                name.text for name in bs_content.find_all("forename")
            ]
        except:
            forenames = ['']
        names += forenames
        try: 
            surname = bs_content.surname.text
        except:
            surname = ''
        names.append(surname)
        person_dict['name'] = ' '.join(names)
        
        # extract keywords
        try:
            keywords = bs_content.find(type="keywords").find_all("term")
            keywords = [
                (term["type"], term.text) for term in keywords
            ]
        except:
            keywords = []
        person_dict['keywords'] = keywords
        
    return person_dict

In [79]:
path = "dcp-data/nameregs/"
files = os.listdir(path)

people_dict = {}
for file in files:
    person_data = extractPersonData(path+file)
    pid = person_data.get("id")
    # some nameregs don't have an id; these aren't going to be any use to us
    if pid:
        people_dict[pid] = person_data
        
# dump to json
with open('people.json', 'w') as jsonfile:
    json.dump(people_dict, jsonfile)